<center>
<img src="../../img/ods_stickers.jpg" />
    
## [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 

Author: [Alexey Natekin](https://www.linkedin.com/in/natekin/), OpenDataScience founder, Machine Learning Evangelist. Translated and edited by [Olga Daykhovskaya](https://www.linkedin.com/in/odaykhovskaya/), [Anastasia Manokhina](https://www.linkedin.com/in/anastasiamanokhina/), [Egor Polusmak](https://www.linkedin.com/in/egor-polusmak/), and [Yuanyuan Pao](https://www.linkedin.com/in/yuanyuanpao/). This material is subject to the terms and conditions of the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license. Free use is permitted for any non-commercial purpose.

# <center> Topic 10. Gradient Boosting

<img src='https://habrastorage.org/web/4a9/edb/082/4a9edb082408442ea47a12b75f19d122.jpg' align='right' width=40%>

Hello ! 

Сегодня мы рассмотрим один из самых популярных и практичных алгоритмов машинного обучения: повышение градиента.

### Краткое содержание статьи
Мы рекомендуем просмотреть эту статью в порядке, описанном ниже, но не стесняйтесь переходить между разделами.

1. Введение и история повышения
    - История градиентной машины
1. алгоритм GBM
    - ML Постановка задачи
    - функциональный градиентный спуск
    - классический алгоритм Фридмана GBM
    - Пошаговый пример алгоритма GBM
1. Функции потери
    - функции потери регрессии
    - Функции классификации потерь
    - Веса
1. Вывод
1. Задание № 10
1. Полезные ресурсы

# 1. Введение и история повышения
Почти все в машинном обучении слышали о повышении градиента. Многие исследователи данных включают этот алгоритм в свой набор инструментов обработки данных из-за хороших результатов, которые он дает по любой заданной (неизвестной) проблеме.
 
Кроме того, XGBoost часто является стандартным рецептом для [победы] (https://github.com/dmlc/xgboost/blob/master/demo/README.md#usecases) [ML соревнования] (http: //blog.kaggle. ком / теги / xgboost /). Он настолько популярен, что идея объединения XGBoosts стала мемом. Кроме того, повышение является важным компонентом [многих рекомендательных систем] (https://en.wikipedia.org/wiki/Learning_to_rank#Practical_usage_by_search_engines); иногда его даже считают [брендом] (https://yandex.com/company/technologies/matrixnet/).
Давайте посмотрим на историю и развитие бустинга.

Boosting родился из [вопроса:] (http://www.cis.upenn.edu/~mkearns/papers/boostnote.pdf), возможно ли получить одну сильную модель из большого количества относительно слабых и простых моделей ?Говоря «слабые модели», мы имеем в виду не простые базовые модели, такие как деревья решений, но модели с низкой производительностью, где плохие показатели немного лучше случайных.[Положительный математический ответ] (http://www.cs.princeton.edu/~schapire/papers/strengthofweak.pdf) на этот вопрос был найден, но потребовалось несколько лет, чтобы разработать полностью функционирующие алгоритмы на основе этого решения, например, AdaBoost. Эти алгоритмы используют жадный подход: во-первых, они строят линейную комбинацию простых моделей (базовых алгоритмов) путем повторного взвешивания входных данных. Затем модель (обычно дерево решений) строится на ранее ошибочно предсказанных объектах, которым теперь присваивается больший вес.<spoiler title = "Подробнее об AdaBoost">Многие курсы машинного обучения изучают AdaBoost - предка GBM (Gradient Boosting Machine). Однако с тех пор, как AdaBoost объединился с GBM, стало очевидно, что AdaBoost - это просто особый вариант GBM.Сам алгоритм имеет очень четкую визуальную интерпретацию и интуицию для определения весов. Давайте посмотрим на следующую проблему классификации игрушек, где мы собираемся разбивать данные между деревьями глубины 1 (также известными как «пни») на каждой итерации AdaBoost. Для первых двух итераций мы имеем следующую картину:

<img src='https://habrastorage.org/web/d28/78f/7ba/d2878f7bad0340fc8002e5ba6d0879a5.jpg' width=70%>

Размер точки соответствует ее весу, который был назначен для неправильного прогноза. На каждой итерации мы видим, что эти веса растут.

<img src = 'https: //habrastorage.org/web/b2b/029/d89/b2b029d898f64bbbb158e15d29595969.png' width = 70%>

псевдокод:
- Инициализировать выборочные веса $ \ Large w_i ^ {(0)} = \ frac {1} {l}, i = 1, \ dots, l $.
- Для всех $ t = 1, \ dots, T $
    * Обучите базовый алгоритм $ \ Large b_t $, пусть $ \ epsilon_t $ будет ошибкой обучения.
    * $ \ Large \ alpha_t = \ frac {1} {2} ln \ frac {1 - \ epsilon_t} {\ epsilon_t} $.
    * Обновление выборочных весов: $ \ Large w_i ^ {(t)} = w_i ^ {(t-1)} e ^ {- \ alpha_t y_i b_t (x_i)}, i = 1, \ dots, l $.
    * Нормализовать веса выборки: $ \ Large w_0 ^ {(t)} = \ sum_ {j = 1} ^ k w_j ^ {(t)}, w_i ^ {(t)} = \ frac {w_i ^ {(t) }} {w_0 ^ {(t)}}, i = 1, \ dots, l $.
- вернуть $ \ sum_t ^ {T} \ alpha_tb_t $


[Здесь] (https://www.youtube.com/watch?v=k4G2VCuOMMg) - более подробный пример AdaBoost, в котором, когда мы выполняем итерацию, мы видим увеличение веса, особенно на границе между классами.

AdaBoost работает хорошо, но [отсутствие] (https://www.cs.princeton.edu/courses/archive/spring07/cos424/papers/boosting-survey.pdf) объяснения того, почему алгоритм успешен, зашило семена сомнения. Некоторые считали это супер-алгоритмом, серебряной пулей, но другие были настроены скептически и полагали, что AdaBoost просто переоснащается.

Проблема переоснащения действительно существовала, особенно когда данные имели сильные выбросы. Поэтому в таких типах проблем AdaBoost был нестабильным. К счастью, несколько профессоров в отделе статистики в Стэнфорде, которые создали Lasso, Elastic Net и Random Forest, начали исследовать алгоритм. В 1999 году Джером Фридман выступил с обобщением разработки алгоритмов повышения - Gradient Boosting (Machine), также известной как GBM. С помощью этой работы Фридман создал статистическую основу для многих алгоритмов, обеспечивающих общий подход повышения для оптимизации в функциональном пространстве.

CART, bootstrap и многие другие алгоритмы были созданы в отделе статистики Стэнфорда. При этом кафедра закрепила их названия в будущих учебниках. Эти алгоритмы очень практичны, и некоторые недавние работы еще не получили широкого распространения. Например, проверьте [glinternet] (https://arxiv.org/abs/1308.2719).

Не так много видеозаписей Фридмана. Хотя с ним очень интересно [интервью] (https://www.youtube.com/watch?v=8hupHmBVvb0) рассказать о создании CART и о том, как они решали проблемы статистики (что похоже на анализ данных и науку о данных). сегодня) более 40 лет назад.

Также есть отличная [лекция] (https://www.youtube.com/watch?v=zBk3PK3g-Fc) от Hastie, ретроспектива по анализу данных от одного из создателей методов, которые мы используем каждый день.

В целом произошел переход от инженерных и алгоритмических исследований к полноценному подходу к созданию и изучению алгоритмов. С математической точки зрения это не большое изменение - мы все еще добавляем (или усиливаем) слабые алгоритмы и расширяем наш ансамбль постепенными улучшениями для частей данных, где модель была неточной. Но на этот раз следующая простая модель не просто построена на повторно взвешенных объектах, но улучшает приближение градиента общей целевой функции. Эта концепция значительно расширяет наши алгоритмы воображения и расширения.

<img src="https://habrastorage.org/webt/h2/v4/k9/h2v4k9r-4yn4jwvwz99fbss4ghi.png" />

## История GBM

Потребовалось более 10 лет после введения GBM, чтобы он стал неотъемлемой частью набора данных по науке.
GBM был расширен для применения к различным статистическим задачам: GLMboost и GAMboost для усиления уже существующих моделей GAM, CoxBoost для кривых выживания и RankBoost и LambdaMART для ранжирования.
Многие реализации GBM также появились под разными именами и на разных платформах: Stochastic GBM, GBDT (деревья решений с градиентным усилением), GBRT (деревья регрессии с градиентным расширением), MART (деревья с множественной аддитивной регрессией) и многие другие. Кроме того, сообщество ОД было очень сегментированным и разобщенным, что затрудняло отслеживание того, насколько широко распространено повышение.

В то же время boosting (повышение) активно использовалось в поисковом рейтинге. Эта проблема была переписана с точки зрения функции потерь, которая штрафует за ошибки в порядке вывода, поэтому стало удобно просто вставить ее в GBM. AltaVista была одной из первых компаний, которые повысили рейтинг. Вскоре идеи распространились на Yahoo, Yandex, Bing и т. Д. Как только это произошло, повышение стало одним из основных алгоритмов, который использовался не только в исследованиях, но и в основных технологиях в промышленности.

<img src = 'https: //habrastorage.org/web/48a/ea4/fff/48aea4fffdbe4e5f9205ba81110e6061.jpg' align = 'right' width = 30%> ML-соревнования, особенно Kaggle, сыграли важную роль в повышении популярности. Теперь у исследователей была общая платформа, где они могли бы конкурировать в различных задачах с данными на большом количестве участников со всего мира. С помощью Kaggle можно было протестировать новые алгоритмы на реальных данных, предоставив алгоритмам возможность «сиять», и предоставить полную информацию в обмене результатами работы модели между наборами данных о конкуренции. Это именно то, что произошло с повышением, когда он был использован в [Kaggle] (http://blog.kaggle.com/2011/12/21/score-xavier-conort-on-coming-second-in-give-me- some-credit /) (проверьте интервью с победителями Kaggle, начиная с 2011 года, которые в основном использовали бустинг). Библиотека [XGBoost] (https://github.com/dmlc/xgboost) быстро завоевала популярность после своего появления. XGBoost не новый, уникальный алгоритм; это просто чрезвычайно эффективная реализация классического GBM с дополнительной эвристикой.

Этот алгоритм прошел очень типичный путь для алгоритмов ML сегодня: математические проблемы и алгоритмические поделки к успешным практическим применениям и массовому внедрению спустя годы после его появления.

# 2. Алгоритм GBM
### ML постановка проблемы

Мы собираемся решить проблему аппроксимации функций в условиях общего обучения. У нас есть набор функций $ \ large x $ и целевые переменные $ \ large y, \ large \ left \ {(x_i, y_i) \ right \} _ {i = 1, \ ldots, n} $, которые мы используем для восстановить зависимость $ \ large y = f (x) $. Мы восстанавливаем зависимость путем аппроксимации $ \ large \ hat {f} (x) $ и понимания того, какое приближение лучше, когда мы используем функцию потерь $ \ large L (y, f) $, которую мы хотим минимизировать: $ \ большой у \ ок \ шляп {ф} (х), \ большой \ хэт {ф} (х) = \ underset {ф (х)} {\ арг \ мин} \ L (у, ф (х)) $

<img src = '.. / .. / img / topic10_help_with_func.png' align = 'center'>

На данный момент мы не делаем никаких предположений относительно типа зависимости $ \ large f (x) $, модели нашего приближения $ \ large \ hat {f} (x) $ или распределения целевой переменной ( $ \ large y $). Мы только ожидаем, что функция $ \ large L (y, f) $ дифференцируема. Наша формула очень общая; давайте определим это для конкретного набора данных со средним значением $ \ large \ hat {f} (x) $. Наше выражение для минимизации потери данных заключается в следующем:
$ \ large \ hat {f} (x) = \ underset {f (x)} {\ arg \ min} \ \ mathbb {E} _ {x, y} [L (y, f (x))]] $

К сожалению, число функций $ \ large f (x) $ не просто большое, но его функциональное пространство бесконечномерно. Вот почему для нас приемлемо ограничить пространство поиска некоторым семейством функций $ \ large f (x, \ theta), \ theta \ in \ mathbb {R} ^ d $. Это значительно упрощает задачу, потому что теперь у нас есть решаемая оптимизация значений параметров:
$ \ large \ hat {f} (x) = f (x, \ hat {\ theta}), \\
\ large \ hat {\ theta} = \ underset {\ theta} {\ arg \ min} \ \ mathbb {E} _ {x, y} [L (y, f (x, \ theta))]] $

Простых аналитических решений для нахождения оптимальных параметров $ \ large \ hat {\ theta} $ часто не существует, поэтому параметры обычно аппроксимируются итеративно. Для начала мы запишем эмпирическую функцию потерь $ \ large L _ {\ theta} (\ hat {\ theta}) $, которая позволит нам оценивать наши параметры, используя наши данные. Кроме того, давайте выпишем наше приближение $ \ large \ hat {\ theta} $ для количества итераций $ \ large M $ в виде суммы:
$ \ large \ hat {\ theta} = \ sum_ {i = 1} ^ M \ hat {\ theta_i}, \\
\ large L _ {\ theta} (\ hat {\ theta}) = \ sum_ {i = 1} ^ N L (y_i, f (x_i, \ hat {\ theta}))) $

Затем остается только найти подходящий итерационный алгоритм для минимизации $ \ large L _ {\ theta} (\ hat {\ theta}) $. Градиентный спуск является самым простым и наиболее часто используемым вариантом. Мы определяем градиент как $ \ large \ nabla L _ {\ theta} (\ hat {\ theta}) $ и добавляем к нему наши итерационные оценки $ \ large \ hat {\ theta_i} $ (так как мы минимизируем потери, мы добавить знак минус). Наш последний шаг - инициализация нашего первого приближения $ \ large \ hat {\ theta_0} $ и выбор количества итераций $ \ large M $. Давайте рассмотрим шаги для этого неэффективного и наивного алгоритма аппроксимации $ \ large \ hat {\ theta} $:

1. Определите начальное приближение параметров $ \ large \ hat {\ theta} = \ hat {\ theta_0} $
2. Для каждой итерации $ \ large t = 1, \ dots, M $ повторите шаги 3-7:
1. Вычислить градиент функции потерь $ \ large \ nabla L _ {\ theta} (\ hat {\ theta}) $ для текущего приближения $ \ large \ hat {\ theta} $
$ \ large \ nabla L _ {\ theta} (\ hat {\ theta}) = \ left [\ frac {\ частичный L (y, f (x, \ theta))}} {\ частичный \ theta} \ right] _ {\ theta = \ hat {\ theta}} $
2. Установите текущую итеративную аппроксимацию $ \ large \ hat {\ theta_t} $ на основе рассчитанного градиента.
$ \ large \ hat {\ theta_t} \ leftarrow - \ nabla L _ {\ theta} (\ hat {\ theta}) $
3. Обновить аппроксимацию параметров $ \ large \ hat {\ theta} $:
$ \ large \ hat {\ theta} \ leftarrow \ hat {\ theta} + \ hat {\ theta_t} = \ sum_ {i = 0} ^ t \ hat {\ theta_i} $
3. Сохраните результат приближения $ \ large \ hat {\ theta} $
$ \ large \ hat {\ theta} = \ sum_ {i = 0} ^ M \ hat {\ theta_i} $
4. Используйте найденную функцию $ \ large \ hat {f} (x) = f (x, \ hat {\ theta}) $

<img src = 'https: //habrastorage.org/web/2b5/5d6/90d/2b55d690d99e4ec0976b360aae6ce4df.jpg' align = 'center'>

### Функциональный градиентный спуск

Давайте на секунду представим, что мы можем выполнить оптимизацию в функциональном пространстве и итеративно искать приближения $ \ large \ hat {f} (x) $ в качестве самих функций. Мы будем выражать наше приближение в виде суммы дополнительных улучшений, каждое из которых является функцией. Для удобства сразу начнем с суммы из начального приближения $ \ large \ hat {f_0} (x) $:
$$ \ large \ hat {f} (x) = \ sum_ {i = 0} ^ M \ hat {f_i} (x) $$

Ничего еще не произошло; мы только решили, что будем искать наше приближение $ \ large \ hat {f} (x) $ не как большую модель с большим количеством параметров (как, например, нейронная сеть), а как сумму функций, делая вид, что мы двигаться в функциональном пространстве.

Чтобы выполнить эту задачу, нам нужно ограничить наш поиск некоторым семейством функций $ \ large \ hat {f} (x) = h (x, \ theta) $. Здесь есть несколько вопросов - во-первых, сумма моделей может быть более сложной, чем любая модель из этого семейства; во-вторых, общая цель все еще в функциональном пространстве. Отметим, что на каждом шаге нам нужно будет выбрать оптимальный коэффициент $ \ large \ rho \ in \ mathbb {R} $. Для шага $ \ large t $ проблема заключается в следующем:
$$ \ large \ hat {f} (x) = \ sum_ {i = 0} ^ {t-1} \ hat {f_i} (x), \\
\ large (\ rho_t, \ theta_t) = \ underset {\ rho, \ theta} {\ arg \ min} \ \ mathbb {E} _ {x, y} [L (y, \ hat {f} (x) + \ rho \ cdot h (x, \ theta))], \\
\ large \ hat {f_t} (x) = \ rho_t \ cdot h (x, \ theta_t) $$

Здесь происходит волшебство. Мы определили все наши цели в общих чертах, как если бы мы могли обучить любой тип модели $ \ large h (x, \ theta) $ для любого типа функций потерь $ \ large L (y, f (x, \) тета)) $. На практике это чрезвычайно сложно, но, к счастью, есть простой способ решить эту задачу.

Зная выражение градиента функции потерь, мы можем рассчитать ее значение по нашим данным. Итак, давайте обучим модели так, чтобы наши прогнозы были более коррелированными с этим градиентом (со знаком минус). Другими словами, мы будем использовать наименьшие квадраты, чтобы исправить прогнозы с этими остатками. Для задач классификации, регрессии и ранжирования мы минимизируем квадратную разницу между псевдо-остатками $ \ large r $ и нашими прогнозами. Для шага $ \ large t $ конечная проблема выглядит следующим образом:
$$ \ large \ hat {f} (x) = \ sum_ {i = 0} ^ {t-1} \ hat {f_i} (x), \\
\ large r_ {it} = - \ left [\ frac {\ частичный L (y_i, f (x_i))}} {\ частичный f (x_i)} \ right] _ {f (x) = \ hat {f} ( x)}, \ quad \ mbox {for} i = 1, \ ldots, n, \\
\ large \ theta_t = \ underset {\ theta} {\ arg \ min} \ \ sum_ {i = 1} ^ {n} (r_ {it} - h (x_i, \ theta)) ^ 2, \\
\ large \ rho_t = \ underset {\ rho} {\ arg \ min} \ \ sum_ {i = 1} ^ {n} L (y_i, \ hat {f} (x_i) + \ rho \ cdot h (x_i, \ theta_t)) $$

<img src = '.. / .. / img / topic10_regression_for_everybody.jpg' align = 'center' width = 60%>

### Классический алгоритм GBM Фридмана

Теперь мы можем определить классический алгоритм GBM, предложенный Джеромом Фридманом в 1999 году. Это контролируемый алгоритм, который имеет следующие компоненты:

- набор данных $ \ large \ left \ {(x_i, y_i) \ right \} _ {i = 1, \ ldots, n} $;
- число итераций $ \ large M $;
- выбор функции потерь $ \ large L (y, f) $ с заданным градиентом;
- выбор семейства функций базовых алгоритмов $ \ large h (x, \ theta) $ с процедурой обучения;
- дополнительные гиперпараметры $ \ large h (x, \ theta) $ (например, в деревьях решений, глубина дерева);

Осталось только начальное приближение $ \ large f_0 (x) $. Для простоты, для начального приближения используется постоянное значение $ \ large \ gamma $. Значение константы, а также оптимальный коэффициент $ \ large \ rho $, идентифицируются с помощью бинарного поиска или другого алгоритма поиска строки по начальной функции потерь (не градиент). Итак, наш алгоритм GBM описан следующим образом:

1. Инициализируйте GBM с постоянным значением $ \ large \ hat {f} (x) = \ hat {f} _0, \ hat {f} _0 = \ gamma, \ gamma \ in \ mathbb {R} $
$ \ large \ hat {f} _0 = \ underset {\ gamma} {\ arg \ min} \ \ sum_ {i = 1} ^ {n} L (y_i, \ gamma) $
2. Для каждой итерации $ \ large t = 1, \ dots, M $, повторите:
1. Рассчитать псевдо-остатки $ \ large r_t $
$ \ large r_ {it} = - \ left [\ frac {\ частичный L (y_i, f (x_i))}} {\ частичный f (x_i)} \ right] _ {f (x) = \ hat {f} (x)}, \ quad \ mbox {for} i = 1, \ ldots, n $
2. Создайте новый базовый алгоритм $ \ large h_t (x) $ как регрессию на псевдо-невязках $ \ large \ left \ {(x_i, r_ {it}) \ right \} _ {i = 1, \ ldots, n} $
3. Найти оптимальный коэффициент $ \ large \ rho_t $ при $ \ large h_t (x) $ относительно начальной функции потерь
$ \ large \ rho_t = \ underset {\ rho} {\ arg \ min} \ \ sum_ {i = 1} ^ {n} L (y_i, \ hat {f} (x_i) + \ rho \ cdot h (x_i) , \ theta)) $
4. Сохраните $ \ large \ hat {f_t} (x) = \ rho_t \ cdot h_t (x) $
5. Обновить текущее приближение $ \ large \ hat {f} (x) $
$ \ large \ hat {f} (x) \ leftarrow \ hat {f} (x) + \ hat {f_t} (x) = \ sum_ {i = 0} ^ {t} \ hat {f_i} (x) $
3. Составьте окончательную модель GBM $ \ large \ hat {f} (x) $
$ \ large \ hat {f} (x) = \ sum_ {i = 0} ^ M \ hat {f_i} (x) $
4. Покори Kaggle и остальной мир

### Пошаговый пример: как работает GBM

Давайте посмотрим пример того, как работает GBM. В этом игрушечном примере мы восстановим функцию с шумом $ \ large y = cos (x) + \ epsilon, \ epsilon \ sim \ mathcal {N} (0, \ frac {1} {5}), x \ in [ -5,5] $.

<img src = 'https: //habrastorage.org/web/9fe/04d/7ba/9fe04d7ba5a645d49fc6aa3e875c8c41.jpg' align = 'center'>

Это регрессионная проблема с реально значимой целью, поэтому мы решим использовать функцию среднеквадратичной потери ошибок. Мы сгенерируем 300 пар наблюдений и приблизим их к деревьям решений глубины 2. Давайте соединим все, что нам нужно для использования GBM:
- Игрушечные данные $ \ large \ left \ {(x_i, y_i) \ right \} _ {i = 1, \ ldots, 300} $ ✓
- количество итераций $ \ large M = 3 $ ✓;
- Среднеквадратичная функция потери ошибок $ \ large L (y, f) = (y-f) ^ 2 $ ✓
- Градиент потери $ \ large L (y, f) = L_2 $ - это просто остатки $ \ large r = (y - f) $ ✓;
- Деревья решений как базовые алгоритмы $ \ large h (x) $ ✓;
- гиперпараметры деревьев решений: глубина деревьев равна 2 ✓;

Для среднеквадратичной ошибки обе инициализации $ \ large \ gamma $ и коэффициенты $ \ large \ rho_t $ просты. Мы инициализируем GBM со средним значением $ \ large \ gamma = \ frac {1} {n} \ cdot \ sum_ {i = 1} ^ n y_i $ и установим все коэффициенты $ \ large \ rho_t $ в 1.

Мы запустим GBM и нарисуем два типа графиков: текущее приближение $ \ large \ hat {f} (x) $ (синий график) и каждое дерево $ \ large \ hat {f_t} (x) $, построенное на его псевдо- остатки (зеленый график). Номер графика соответствует номеру итерации:

<img src = 'https: //habrastorage.org/web/edb/328/98a/edb32898ad014d8d95782759d11f63fb.png' align = 'center'>

Ко второй итерации наши деревья восстановили базовый вид функции. Однако на первой итерации мы видим, что алгоритм построил только «левую ветвь» функции ($ \ large x \ in [-5, -4] $). Это было связано с тем, что нашим деревьям просто не хватало глубины для одновременного построения симметричной ветви, и она фокусировалась на левой ветви с большей ошибкой. Поэтому правая ветвь появилась только после второй итерации.

Остальная часть процесса идет, как и ожидалось - на каждом шаге наши псевдо-остатки уменьшаются, и GBM лучше и лучше аппроксимирует исходную функцию с каждой итерацией. Однако по построению деревья не могут аппроксимировать непрерывную функцию, что означает, что GBM не идеален в этом примере. Чтобы поиграть с приближениями функций GBM, вы можете использовать потрясающую интерактивную демонстрацию в этом блоге, которая называется [Совершенно неправильно] (http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html):

<img src = 'https: //habrastorage.org/web/779/3e0/e66/7793e0e66b7d4871b6391a94cd5d4cf2.jpg' align = 'center'>

# 3. Функции потери

Если мы хотим решить проблему классификации вместо регрессии, что изменится? Нам нужно только выбрать подходящую функцию потерь $ \ large L (y, f) $. Это самый важный момент, который точно определяет, как мы будем оптимизировать и какие характеристики мы можем ожидать в окончательной модели.

Как правило, нам не нужно это изобретать самим - исследователи уже сделали это для нас. Сегодня мы рассмотрим функции потерь для двух наиболее распространенных целей: регрессии $ \ large y \ in \ mathbb {R} $ и бинарной классификации $ \ large y \ in \ left \ {- 1, 1 \ right \} $.

### Функции потери регрессии

Начнем с задачи регрессии для $ \ large y \ in \ mathbb {R} $. Чтобы выбрать подходящую функцию потерь, нам нужно рассмотреть, какое из свойств условного распределения $ \ large (y | x) $ мы хотим восстановить. Наиболее распространенные варианты:

- $ \ large L (y, f) = (y - f) ^ 2 $ a.k.a. $ \ large L_2 $ убыток или гауссовская потеря. Это классическое условное среднее, которое является самым простым и наиболее распространенным случаем. Если у нас нет никакой дополнительной информации или требований для надежности модели, мы можем использовать потерю Гаусса.
- $ \ large L (y, f) = | y - f | $ a.k.a. $ \ large L_1 $ убыток или потеря Лапласа. На первый взгляд эта функция не кажется дифференцируемой, но на самом деле она определяет условную медиану. Медиана, как мы знаем, устойчива к выбросам, поэтому в некоторых случаях эта функция потерь лучше. Наказание за большие вариации не такое тяжелое, как в $ \ large L_2 $.
- $ \ large \ begin {уравнение} L (y, f) = \ left \ {\ begin {array} {@ {} ll @ {}} (1 - \ alpha) \ cdot | y - f |, & \ text {if} \ yf \ leq 0 \\ \ alpha \ cdot | y - f |, & \ text {if} \ yf> 0 \ end {array} \ right. \ end {уравнение}, \ alpha \ in (0,1)
$ a.k.a. $ \ large L_q $ убыток или квантильный убыток. Вместо медианы используются квантили. Например, $ \ large \ alpha = 0.75 $ соответствует 75% -в квантилю. Мы можем видеть, что эта функция асимметрична и наказывает наблюдения, которые находятся справа от определенного квантиля.

<img src = 'https: //habrastorage.org/web/6d5/e3a/09c/6d5e3a09c703491b947fde851e412ac0.png' width = 60%>

Давайте использовать функцию потерь $ \ large L_q $ на наших данных. Целью является восстановление условного 75% -квантов косинуса. Давайте соединяем все вместе для GBM:
- Игрушечные данные $ \ large \ left \ {(x_i, y_i) \ right \} _ {i = 1, \ ldots, 300} $ ✓
- количество итераций $ \ large M = 3 $ ✓;
- Функция потерь для квантилей $ \ large \ begin {уравнение} L_ {0.75} (y, f) = \ left \ {
\ begin {array} {@ {} ll @ {}} 0.25 \ cdot | y - f |, & \ text {if} \ yf \ leq 0 \\ 0.75 \ cdot | y - f |, & \ text {if } \ yf> 0 \ end {array} \ right. \ end {уравнение} $ ✓;
- Градиентная $ \ large L_ {0.75} (y, f) $ - функция, взвешенная по $ \ large \ alpha = 0.75 $. Мы собираемся обучить древовидную модель для классификации:
$ \ large r_ {i} = - \ left [\ frac {\ частичный L (y_i, f (x_i))}} {\ частичный f (x_i)} \ right] _ {f (x) = \ hat {f} (x)} = $
$ \ large = \ alpha I (y_i> \ hat {f} (x_i)) - (1 - \ alpha) I (y_i \ leq \ hat {f} (x_i)), \ quad \ mbox {for} i = 1, \ ldots, 300 $ ✓;
- Дерево решений как базовый алгоритм $ \ large h (x) $ ✓;
- гиперпараметр деревьев: глубина = 2 ✓;

Для нашего начального приближения мы возьмем необходимый квантиль $ \ large y $. Однако мы ничего не знаем об оптимальных коэффициентах $ \ large \ rho_t $, поэтому будем использовать стандартный поиск строк. Результаты следующие:

<img src = 'https: //habrastorage.org/web/0e6/7dd/614/0e67dd614076499e91c8c4238457ae4d.png' align = 'center'>

Мы можем наблюдать, что на каждой итерации $ \ large r_ {i} $ принимает только 2 возможных значения, но GBM все еще может восстановить нашу начальную функцию.

Общие результаты GBM с функцией квантильных потерь такие же, как результаты с квадратичной функцией потерь, смещенной на $ \ large \ приблизительно 0.135 $. Но если бы мы использовали 90-процентный квантиль, у нас не было бы достаточно данных из-за того, что классы стали бы несбалансированными. Мы должны помнить это, когда имеем дело с нестандартными проблемами.

* «Несколько слов о функциях потери регрессии» *

Для задач регрессии было разработано много функций потерь, некоторые из которых имеют дополнительные свойства. Например, они могут быть надежными, как в [функции потерь Хьюбера] (https://en.wikipedia.org/wiki/Huber_loss). Для небольшого числа выбросов функция потерь работает как $ \ large L_2 $, но после определенного порога функция меняется на $ \ large L_1 $. Это позволяет уменьшить влияние выбросов и сосредоточиться на общей картине.

Мы можем проиллюстрировать это следующим примером. Данные генерируются из функции $ \ large y = \ frac {sin (x)} {x} $ с добавленным шумом, смесью из нормального распределения и распределения Бернулли. Мы показываем функции на графиках A-D и соответствующие GBM на F-H (график E представляет начальную функцию):

<img src = 'https: //habrastorage.org/web/130/05b/222/13005b222e8a4eb68c3936216c05e276.jpg' align = 'center'> [Оригинальный размер] (https://habrastorage.org/web/130/05b/ 222 / 13005b222e8a4eb68c3936216c05e276.jpg).


В этом примере мы использовали сплайны в качестве базового алгоритма. Видите ли, это не всегда должны быть деревья для повышения?

### Классификация функций потерь

Теперь давайте посмотрим на проблему двоичной классификации $ \ large y \ in \ left \ {- 1, 1 \ right \} $. Мы видели, что GBM может даже оптимизировать недифференцируемые функции потерь. Технически эту проблему можно решить с помощью регрессии $ \ large L_2 $ loss, но это было бы неправильно.

Распределение целевой переменной требует от нас использования логарифмического сходства, поэтому нам нужно иметь разные функции потерь для целей, умноженные на их прогнозы: $ \ large y \ cdot f $. Наиболее распространенные варианты будут следующими:

- $ \ large L (y, f) = log (1 + exp (-2yf)) $ a.k.a. Потери логистики или потери Бернулли. У этого есть интересное свойство, которое наказывает даже правильно спрогнозированные классы, что помогает не только помочь оптимизировать потери, но также и раздвинуть классы дальше, даже если все классы спрогнозированы правильно.
- $ \ large L (y, f) = exp (-yf) $ a.k.a. Потеря AdaBoost. Классическая AdaBoost эквивалентна GBM с этой функцией потерь. Концептуально эта функция очень похожа на логистические потери, но она имеет большую экспоненциальную штрафность, если прогноз неверен.

<img src = 'https: //habrastorage.org/web/bf5/9de/dcf/bf59dedcfd9d49b18e89ce342b09ce69.png' width = 60%>

Давайте сгенерируем новые игрушечные данные для нашей задачи классификации. За основу мы возьмем наш шумный косинус и будем использовать функцию знака для классов целевой переменной. Наши данные игрушек выглядят следующим образом (для ясности добавлен джиттер-шум):

<img src = 'https: //habrastorage.org/web/e72/513/78b/e7251378bf6d459ab1aeea7a1f1996a1.jpg'>


Мы будем использовать логистические потери, чтобы искать то, что мы действительно повышаем. Итак, еще раз, мы собрали то, что мы будем использовать для GBM:
- Игрушечные данные $ \ large \ left \ {(x_i, y_i) \ right \} _ {i = 1, \ ldots, 300}, y_i \ in \ left \ {- 1, 1 \ right \} $ ✓
- количество итераций $ \ large M = 3 $ ✓;
- Логистические потери как функция потерь, градиент которых рассчитывается следующим образом:
$ \ large r_ {i} = \ frac {2 \ cdot y_i} {1 + exp (2 \ cdot y_i \ cdot \ hat {f} (x_i))}, \ quad \ mbox {for} i = 1, \ лдоты, 300 $ ✓;
- Деревья решений как базовые алгоритмы $ \ large h (x) $ ✓;
- гиперпараметры деревьев решений: глубина дерева равна 2 ✓;

На этот раз инициализация алгоритма немного сложнее. Во-первых, наши классы не сбалансированы (63% против 37%). Во-вторых, не существует известной аналитической формулы для инициализации нашей функции потерь, поэтому мы должны искать $ \ large \ hat {f_0} = \ gamma $ через поиск:

<img src = 'https: //habrastorage.org/web/f8a/054/702/f8a05470271448d9bc0d4dc3e524a571.png' width = 60%>


Наше оптимальное начальное приближение составляет около -0,273. Вы могли догадаться, что он был отрицательным, потому что все более выгодно прогнозировать как самый популярный класс, но нет точной формулы для точного значения. Теперь давайте наконец запустим GBM и посмотрим, что на самом деле происходит под капотом:

<img src = 'https: //habrastorage.org/web/7b4/ab0/5fa/7b4ab05fa0a543bfad94950e47f91568.png' align = 'center'>

Алгоритм успешно восстановил разделение между нашими классами. Вы можете видеть, как «нижние» области разделяются, потому что деревья более уверены в правильном предсказании отрицательного класса и как формируются два шага смешанных классов. Ясно, что у нас есть много правильно классифицированных наблюдений и некоторое количество наблюдений с большими ошибками, которые появились из-за шума в данных.

### Веса

Иногда возникает ситуация, когда нам нужна более конкретная функция потерь для нашей проблемы. Например, в финансовых временных рядах мы можем захотеть придать больший вес большим движениям во временных рядах; для прогнозирования оттока более полезно предсказать отток клиентов с высоким LTV (или пожизненной стоимостью: сколько денег принесет клиент в будущем).

<img src = 'https: //habrastorage.org/web/0c0/ad0/3a4/0c0ad03a4c4b46bfa5bcd5101678c9c4.jpg' align = 'center'>

Статистический воин должен был бы изобрести свою собственную функцию потерь, выписать для нее градиент (для более эффективного обучения, включая гессиан) и тщательно проверить, удовлетворяет ли эта функция требуемым свойствам. Тем не менее, существует высокая вероятность ошибиться где-то, столкнуться с вычислительными трудностями и потратить слишком много времени на исследования.

Вместо этого был изобретен очень простой инструмент (который редко вспоминается на практике): взвешивание наблюдений и назначение весовых функций. Простейшим примером такого взвешивания является установка весов для баланса классов. В общем, если мы знаем, что некоторое подмножество данных, как во входных переменных $ \ large x $, так и в целевой переменной $ \ large y $, имеет большее значение для нашей модели, то мы просто присваиваем им больший вес $ \ большой w (x, y) $. Основная цель - выполнить общие требования к весам:
$$ \ large w_i \ in \ mathbb {R}, \\
\ large w_i \ geq 0 \ quad \ mbox {for} i = 1, \ ldots, n, \\
\ large \ sum_ {i = 1} ^ n w_i> 0 $$

Весовые коэффициенты могут значительно сократить время, затрачиваемое на настройку функции потерь для решаемой нами задачи, а также стимулировать эксперименты со свойствами целевых моделей. Назначение этих весов полностью зависит от творчества. Мы просто добавляем скалярные веса:
$$ \ large L_ {w} (y, f) = w \ cdot L (y, f), \\
\ large r_ {it} = - w_i \ cdot \ left [\ frac {\ частичное L (y_i, f (x_i))} {{частичное f (x_i)} \ right] _ {f (x) = \ hat { f} (x)}, \ quad \ mbox {for} i = 1, \ ldots, n $$

Ясно, что для произвольных весов мы не знаем статистических свойств нашей модели. Часто привязка весов к значениям $ \ large y $ может быть слишком сложной. Например, использование весов, пропорциональных $ \ large | y | $ в функции потерь $ \ large L_1 $, не эквивалентно потере $ \ large L_2 $, поскольку градиент не будет учитывать значения самих прогнозов: $ \ large \ hat {f} (x) $.

Мы упоминаем все это, чтобы лучше понять наши возможности. Давайте создадим очень экзотические веса для наших игрушечных данных. Мы определим сильно асимметричную весовую функцию следующим образом:
$$ \ large \ begin {уравнение} w (x) = \ left \ {\ begin {array} {@ {} ll @ {}} 0,1, & \ text {if} \ x \ leq 0 \\ 0,1 + | cos (x) |, & \ text {if} \ x> 0 \ end {array} \ right. \ end {уравнение} $$

<img src = 'https: //habrastorage.org/web/8c2/1b1/aa4/8c21b1aa47134f7aa46b15ef910369b2.png' align = 'center'>

С этими весами мы ожидаем получить два свойства: меньше детализации для отрицательных значений $ \ large x $ и форму функции, аналогичную начальному косинусу. Мы берем другие настройки GBM из нашего предыдущего примера с классификацией, включающей поиск линии для оптимальных коэффициентов. Давайте посмотрим, что у нас есть:

<img src = 'https: //habrastorage.org/web/afc/cca/72a/afccca72a0774990b685de37b0fe9d9f.png' align = 'center'>

Мы достигли ожидаемого результата. Во-первых, мы видим, насколько сильно отличаются псевдо-остатки; на начальной итерации они выглядят почти как первоначальный косинус. Во-вторых, левая часть графика функции часто игнорировалась в пользу правой, которая имела больший вес. В-третьих, функция, которую мы получили на третьей итерации, получила достаточно внимания и стала похожа на первоначальный косинус (также начала слегка перегоняться).

Веса - это мощный, но рискованный инструмент, который мы можем использовать для управления свойствами нашей модели. Если вы хотите оптимизировать свою функцию потерь, стоит сначала попытаться решить более простую задачу, но добавить веса к наблюдениям на ваше усмотрение.

# 4. Вывод

Сегодня мы изучили теорию градиенртного "boosting". GBM - это не просто определенный алгоритм, а общая методология построения ансамблей моделей. Кроме того, эта методология является достаточно гибкой и расширяемой - можно обучить большое количество моделей, принимая во внимание различные функции потерь с различными весовыми функциями.

Практика и соревнования ML показывают, что в стандартных задачах (за исключением изображения, аудио и очень разреженных данных) GBM часто является наиболее эффективным алгоритмом (не говоря уже о стековых и высокоуровневых ансамблях, где GBM почти всегда является их частью). ). Кроме того, существует множество адаптаций GBM [для обучения усилению] (https://arxiv.org/abs/1603.04119) (Minecraft, ICML 2016). Кстати, алгоритм Виолы-Джонса, который все еще используется в компьютерном зрении, [основан на AdaBoost] (https://en.wikipedia.org/wiki/Viola%E2%80%93Jones_object_detection_framework#Learning_algorithm).

В этой статье мы намеренно пропустили вопросы, касающиеся регуляризации, стохастичности и гиперпараметров GBM. Не случайно мы использовали небольшое количество итераций $ \ large M = 3 $. Если бы мы использовали 30 деревьев вместо 3 и обучили GBM, как описано, результат не был бы таким предсказуемым:

<img src = '.. / .. / img / topic10_good_fit.png' align = 'center' width = 60%>
<img src = '.. / .. / img / topic10_overfitting.png' align = 'center' width = 60%>


<img src = 'https: //habrastorage.org/web/27f/0f5/3be/27f0f53be9424cb1afaffb9a0e32909f.jpg' align = 'center'> [http://arogozhnikov.github.io/2016/07/05/gradient_boosting_playground. HTML] (http://arogozhnikov.github.io/2016/07/05/gradient_boosting_playground.html)

# 5. Задание № 10
Ваша [задача] (http://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic10_boosting/assignment10_flight_delays_kaggle.ipynb) должна пройти как минимум 2 теста в этом классе Kaggle: [конкуренция] (https) //www.kaggle.com/c/flight-delays-spring-2018). Здесь вам не будут предоставлены подробные инструкции. Мы только дадим вам краткое описание того, как второй тест был достигнут с помощью XGBoost.

# 6. Useful links
- Main course [site](https://mlcourse.ai), [course repo](https://github.com/Yorko/mlcourse.ai), and YouTube [channel](https://www.youtube.com/watch?v=QKTuw4PNOsU&list=PLVlY_7IJCMJeRfZ68eVfEcu-UcN9BbwiX)
- Course materials as a [Kaggle Dataset](https://www.kaggle.com/kashnitsky/mlcourse)
- [Original article](https://statweb.stanford.edu/~jhf/ftp/trebst.pdf) about GBM from Jerome Friedman
- “Gradient boosting machines, a tutorial”, [paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3885826/) by Alexey Natekin, and Alois Knoll
- [Chapter in Elements of Statistical Learning](http://statweb.stanford.edu/~tibs/ElemStatLearn/printings/ESLII_print10.pdf) from Hastie, Tibshirani, Friedman (page 337)
- [Wiki](https://en.wikipedia.org/wiki/Gradient_boosting) article about Gradient Boosting
- [Introduction to boosted trees (Xgboost docs)](https://xgboost.readthedocs.io/en/latest/tutorials/model.html)
- [Video-lecture by Hastie](https://www.youtube.com/watch?v=wPqtzj5VZus) about GBM at h2o.ai conference
- [CatBoost vs. Light GBM vs. XGBoost](https://towardsdatascience.com/catboost-vs-light-gbm-vs-xgboost-5f93620723db) on "Towards Data Science"
- [Benchmarking and Optimization of
Gradient Boosting Decision Tree Algorithms](https://arxiv.org/abs/1809.04559), [XGBoost: Scalable GPU Accelerated Learning](https://arxiv.org/abs/1806.11248) - benchmarking CatBoost, Light GBM, and XGBoost (no 100% winner)